<a href="https://colab.research.google.com/github/jellyfizhi/genAI_experiment/blob/main/genAIExperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "tiiuae/falcon-7b-instruct"
save_path="/Falcon"
def download_LLM(model_name, save_path):
    """
    Downloads a LLM model from Hugging Face and saves it locally.

    Parameters:
        model_name (str): The name of the GPT-2 model to download. Options:
                          - 'gpt2' (small)
                          - 'gpt2-medium'
                          - 'gpt2-large'
                          - 'gpt2-xl'
        save_path (str): Directory to save the model and tokenizer.
    """
    # Load tokenizer and model from Hugging Face
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

    # Save locally
    tokenizer.save_pretrained(save_path)
    model.save_pretrained(save_path)

    print(f"✅ LLM model ('{model_name}') downloaded and saved to '{save_path}'.")

# Example usage
download_LLM(model_name, save_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3405: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the GPT-2 model and tokenizer
def load_LLM(save_path):
    """
    Loads the model and tokenizer from a saved local path.
    """
    tokenizer = AutoTokenizer.from_pretrained(save_path)
    model = AutoModelForCausalLM.from_pretrained(save_path)
    return tokenizer, model

# Function to chat with GPT-2
def chat_with_LLM():
    tokenizer, model = load_LLM(save_path="/Dialo")  # Load the model

    print("🤖 Chatbot: Hello! Type 'exit' to end the chat.")

    while True:
        user_input = input("🧑 You: ")
        if user_input.lower() == "exit":
            print("🤖 Chatbot: Goodbye!")
            break

        # Tokenize input and generate a response
        input_ids = tokenizer.encode(user_input, return_tensors="pt")
        output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

        # Decode and print response
        response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
        print(f"🤖 Chatbot: {response}")


In [1]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   55G   59G  49% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G     0  5.7G   0% /dev/shm
/dev/root       2.0G  1.2G  820M  59% /usr/sbin/docker-init
/dev/sda1        92G   70G   22G  77% /opt/bin/.nvidia
tmpfs           6.4G  3.3M  6.4G   1% /var/colab
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [3]:
!ls / -all

total 464
drwxr-xr-x   1 root root   4096 Apr  1 20:55 .
drwxr-xr-x   1 root root   4096 Apr  1 20:55 ..
lrwxrwxrwx   1 root root      7 Jun 27  2024 bin -> usr/bin
drwxr-xr-x   2 root root   4096 Apr 18  2022 boot
drwxr-xr-x   1 root root   4096 Mar 31 13:40 content
-rw-r--r--   1 root root   4332 Jul 10  2024 cuda-keyring_1.1-1_all.deb
drwxr-xr-x   1 root root   4096 Mar 31 13:53 datalab
drwxr-xr-x   6 root root    460 Apr  1 20:09 dev
drwxr-xr-x   2 root root   4096 Apr  1 20:55 Dialo
-rwxr-xr-x   1 root root      0 Apr  1 20:09 .dockerenv
drwxr-xr-x   1 root root   4096 Apr  1 20:09 etc
drwxr-xr-x   2 root root   4096 Apr  1 20:41 gpt2_model
drwxr-xr-x   2 root root   4096 Apr 18  2022 home
drwxr-xr-x   3 root root   4096 Apr  1 20:09 kaggle
lrwxrwxrwx   1 root root      7 Jun 27  2024 lib -> usr/lib
lrwxrwxrwx   1 root root      9 Jun 27  2024 lib32 -> usr/lib32
lrwxrwxrwx   1 root root      9 Jun 27  2024 lib64 -> usr/lib64
lrwxrwxrwx   1 root root     10 Jun 27  2024 libx32 -> u

In [17]:
!ls -a /usr/local/

.   bin    cuda     cuda-12.5	      etc    include  libexec	  man  sbin   src
..  colab  cuda-12  dist_metrics.pxd  games  lib      LICENSE.md  opt  share


In [10]:
# Run the chatbot
chat_with_LLM()

🤖 Chatbot: Hello! Type 'exit' to end the chat.
🧑 You: hi, how are you
🤖 Chatbot: ?
🧑 You: what?
🤖 Chatbot: ??
🧑 You: i'm confused too
🤖 Chatbot: .
🧑 You: how are you?
🤖 Chatbot: 
🧑 You: i'm your creator
🤖 Chatbot: .


KeyboardInterrupt: Interrupted by user